# 自动编码器

参考资料：

- [Autoencoder Explained](https://www.youtube.com/watch?v=H1AllrJ-_30)
- [Introduction to autoencoders.](https://www.jeremyjordan.me/autoencoders/)
- [L1aoXingyu/pytorch-beginner](https://github.com/L1aoXingyu/pytorch-beginner)
- [An Introduction to Autoencoders: Everything You Need to Know ](https://www.v7labs.com/blog/autoencoders-guide)
- [Auto-Encoder: What Is It? And What Is It Used For? (Part 1)](https://towardsdatascience.com/auto-encoder-what-is-it-and-what-is-it-used-for-part-1-3e5c6f017726)

自（动）编码器是一种无监督学习技术，我们利用神经网络来完成表征学习任务。具体来说，我们将设计一个神经网络架构，以便在网络中施加bottleneck，迫使原始输入的压缩知识表示。如果输入特征彼此独立，则这种压缩和随后的重建将是一项非常艰巨的任务。但是，如果数据中存在某种结构（即输入特征之间的相关性），则可以学习并因此在强制输入通过网络瓶颈时利用该结构。

![](img/Screen-Shot-2018-03-06-at-3.17.13-PM.png)

如上图所示，我们可以将一个未标记的数据集构建为一个监督学习问题，其任务是输出 $\hat x$，重建原始输入 x。该网络可以通过最小化重建误差 L(x,$\hat x$) 来训练，L(x,$\hat x$) 衡量我们的原始输入和随后的重建之间的差异。瓶颈是我们网络设计的一个关键属性；在不存在信息瓶颈的情况下，我们的网络可以轻松学会通过将这些值传递到网络（如下所示）来简单地记住输入值。

![](img/Screen-Shot-2018-03-06-at-6.09.05-PM.png)

瓶颈限制了可以遍历整个网络的信息量，迫使对输入数据进行学习压缩。

注意：事实上，如果我们要构建一个线性网络（即在每一层不使用非线性激活函数），我们将观察到与 PCA 中观察到的类似的降维。

理想的自动编码器模型会权衡以下两点：

- 对输入足够敏感，可以准确地构建重建。
- 对输入足够不敏感，以至于模型不会简单地记住或过度拟合训练数据。

这种权衡迫使模型只保留重建输入所需的数据变化，而不保留输入中的冗余。在大多数情况下，这涉及构建一个损失函数，其中一项鼓励我们的模型对输入敏感（即重建损失L(x,$\hat x$)），第二项不鼓励记忆/过拟合（即添加正则化） .

$${\cal L}\left( {x,\hat x} \right) + regularizer$$

我们通常会在正则化项之前添加一个缩放参数，以便我们可以调整两个目标之间的权衡，类似于惩罚系数。

这里将讨论一些用于加这两个约束并调整权衡的标准自动编码器架构；在后续更多神经网络的介绍中，会进一步讨论更复杂的如变分自编码器等，它建立在此处讨论的概念之上，以提供更强大的模型。

In [1]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [3]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [2]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [5]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

0it [00:00, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz


0it [00:00, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz


0it [00:00, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...


C:\Users\hust2\miniconda3\envs\HDB\lib\site-packages\torchvision\datasets\mnist.py:335: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed).view(length, num_rows, num_cols)


Done!


In [6]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [11]:
model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print("epoch [{}/{}], loss:{:.4f}".format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './data/sim_autoencoder.pth')

epoch [1/100], loss:0.1912
epoch [2/100], loss:0.1683
epoch [3/100], loss:0.1613
epoch [4/100], loss:0.1641
epoch [5/100], loss:0.1461
epoch [6/100], loss:0.1451
epoch [7/100], loss:0.1489
epoch [8/100], loss:0.1435
epoch [9/100], loss:0.1420
epoch [10/100], loss:0.1317
epoch [11/100], loss:0.1495
epoch [12/100], loss:0.1478
epoch [13/100], loss:0.1480
epoch [14/100], loss:0.1314
epoch [15/100], loss:0.1418
epoch [16/100], loss:0.1376
epoch [17/100], loss:0.1234
epoch [18/100], loss:0.1244
epoch [19/100], loss:0.1373
epoch [20/100], loss:0.1413
epoch [21/100], loss:0.1235
epoch [22/100], loss:0.1385
epoch [23/100], loss:0.1323
epoch [24/100], loss:0.1253
epoch [25/100], loss:0.1430
epoch [26/100], loss:0.1299
epoch [27/100], loss:0.1251
epoch [28/100], loss:0.1310
epoch [29/100], loss:0.1312
epoch [30/100], loss:0.1207
epoch [31/100], loss:0.1271
epoch [32/100], loss:0.1284
epoch [33/100], loss:0.1174
epoch [34/100], loss:0.1266
epoch [35/100], loss:0.1182
epoch [36/100], loss:0.1336
e

## 不完全自编码器

构建自编码器的最简单架构是限制网络隐藏层中存在的节点数量，限制可以流经网络的信息量。通过根据重构误差惩罚网络，我们的模型可以学习输入数据的最重要属性，以及如何从“编码”状态最好地重构原始输入。理想情况下，这种编码将学习和描述输入数据的潜在属性。

![](img/Screen-Shot-2018-03-07-at-8.24.37-AM.png)

由于神经网络能够学习非线性关系，因此可以将其视为 PCA 的更强大（非线性）泛化。 PCA 试图发现描述原始数据的低维超平面，而自编码器能够学习非线性流形（流形用简单的术语定义为连续的、不相交的表面）。这两种方法之间的区别如下所示。

![](img/Screen-Shot-2018-03-07-at-8.52.21-AM.png)

对于更高维的数据，自编码器能够学习数据（流形）的复杂表示，该表示可用于描述较低维的观察，并相应地解码到原始输入空间中。

![](img/LinearNonLinear.png)

不完全自编码器没有明确的正则化项——我们只是根据重建损失训练我们的模型。因此，我们确保模型不会记住输入数据的唯一方法是确保我们已经充分限制了隐藏层中的节点数量。

对于深度自动编码器，我们还必须了解编码器和解码器模型的容量。即使“瓶颈层”只是一个隐藏节点，我们的模型仍然有可能记住训练数据，前提是编码器和解码器模型有足够的能力学习一些可以将数据映射到索引的任意函数。

鉴于我们希望我们的模型发现我们数据中的潜在属性，确保自动编码器模型不是简单地学习一种记忆训练数据的有效方法是很重要的。与监督学习问题类似，我们可以对网络采用各种形式的正则化，以鼓励良好的泛化特性。

## 稀疏自编码器

稀疏自动编码器为我们提供了一种引入信息瓶颈的替代方法，而无需减少隐藏层的节点数量。它将构建损失函数来惩罚层内的激活。对于任何给定的观察，将鼓励网络学习仅依赖于激活少量神经元的编码和解码。

也就是说，损失函数有一项计算激活的神经元个数，并给一个正比于该数量的惩罚，这个惩罚被称为 sparsity 函数。值得注意的是，这是一种不同的正则化方法，因为通常是对网络的权重进行正则化，而不是激活数。

下面是一个通用的稀疏自编码器，其中节点的不透明度与激活级别相对应。重要的是要注意，激活的训练模型的各个节点是依赖于数据的，不同的输入将导致通过网络激活不同的节点。

![](img/Screen-Shot-2018-03-07-at-1.50.55-PM.png)

这一事实的一个结果是我们允许我们的网络使各个隐藏层节点对输入数据的特定属性敏感。不完全自编码器将使用整个网络进行每次观察，而稀疏自编码器将被迫根据输入数据选择性地激活网络区域。因此，我们限制了网络记忆输入数据的能力，而没有限制网络从数据中提取特征的能力。这允许我们分别考虑网络的潜在状态表示和正则化，以便我们可以根据给定数据上下文的意义选择潜在状态表示（即编码维度），同时通过稀疏约束强加正则化.

我们可以通过两种主要方式来施加这种稀疏性约束；两者都涉及衡量每个训练批次的隐藏层激活数，并向损失函数添加一些项以惩罚过度激活：

1. L1 正则化：我们可以在我们的损失函数中添加一个项，该项会惩罚 h 层中用于观察 i 的激活向量 a 的绝对值，由调整参数 λ 缩放。$${\cal L}\left( {x,\hat x} \right) +  \lambda \sum\limits_i {\left| {a_i^{\left( h \right)}} \right|}$$
2. KL-Divergence：本质上，KL-divergence 是衡量两个概率分布之间的差异。我们可以定义一个稀疏参数 ρ，它表示一个神经元在一组样本上的平均激活。该期望值可以计算为 ${{\hat \rho }_ j} = \frac{1}{m}\sum\limits_{i} {\left[ {a_i^{\left( h \right)}\left( x \right)} \right]}$，其中下标 j 表示层 h 中的特定神经元，将 m 个训练观察的激活值相加，分别表示为 x。从本质上讲，通过限制一个神经元在一组样本上的平均激活，我们鼓励神经元只对观察的一个子集进行激活。我们可以将 ρ 描述为伯努利随机变量分布，这样我们就可以利用 KL 散度（在下面进行扩展）将理想分布 ρ 与在所有隐藏层节点 ρ^ 上观察到的分布进行比较。$${\cal L}\left( {x,\hat x} \right) + \sum\limits_{j} {KL\left( {\rho ||{{\hat \rho }_ j}} \right)}$$ 

注意：伯努利分布是“随机变量的概率分布，其取值为 1 的概率为 p，取值为 0 的概率为 q=1−p”。这与确定神经元触发的概率非常吻合。

两个伯努利分布之间的 KL 散度可以写成 $\sum\limits_{j = 1}^{{l^{\left( h \right)}}} {\rho \log \frac{\rho }{{{{\hat \rho }_ j}}}}  + \left( {1 - \rho } \right)\log \frac{{1 - \rho }}{{1 - {{\hat \rho }_ j}}}$。对于 ρ=0.2 的理想分布，此损失项在下面可视化，对应于此时的最小（零）惩罚。

![](img/KLPenaltyExample-1.png)

## 去噪自编码器

到目前为止，我已经讨论了训练输入和输出相同的神经网络的概念，我们的模型的任务是在通过某种信息瓶颈的同时尽可能接近地再现输入。回想一下，我提到我们希望我们的自动编码器足够敏感以重新创建原始观察，但对训练数据足够不敏感，以便模型学习可泛化的编码和解码。开发可泛化模型的另一种方法是稍微损坏输入数据，但仍将未损坏的数据保留为我们的目标输出。

![](img/Screen-Shot-2018-03-09-at-10.20.44-AM.png)

使用这种方法，我们的模型不能简单地开发一个映射来记忆训练数据，因为我们的输入和目标输出不再相同。相反，该模型学习了一个向量场，用于将输入数据映射到低维流形（回想一下我之前的图形，流形描述了输入数据集中的高密度区域）；如果这个流形准确地描述了自然数据，我们就有效地“消除了”增加的噪音。

![](img/Screen-Shot-2018-03-09-at-10.12.59-PM.png)

上图通过比较 x 的重构与 x 的原始值来可视化描述的向量场。黄点表示添加噪声之前的训练示例。如您所见，该模型已学会将损坏的输入调整为学习的流形。

值得注意的是，这个向量场通常只在模型在训练期间观察到的区域表现良好。在远离自然数据分布的区域，重建误差既大又不总是指向真实分布的方向。

![](img/Screen-Shot-2018-03-10-at-10.17.44-AM.png)

## 收缩自编码器

人们会期望对于非常相似的输入，学习到的编码也会非常相似。我们可以明确地训练我们的模型，通过要求隐藏层激活的导数相对于输入很小。换句话说，对于输入的微小变化，我们仍然应该保持非常相似的编码状态。这与去噪自编码器非常相似，因为这些对输入的小扰动本质上被视为噪声，并且我们希望我们的模型对噪声具有鲁棒性。换句话说，“去噪自编码器使重构函数（即解码器）抵抗输入的小但有限大小的扰动，而收缩自编码器使特征提取函数（即编码器）抵抗输入的无穷小扰动。”

因为我们明确鼓励我们的模型学习一种编码，其中相似的输入具有相似的编码，所以我们本质上是在强迫模型学习如何将输入的邻域收缩为较小的输出邻域。请注意，对于输入数据的局部邻域，重建数据的斜率（即导数）基本上为零。

![](img/Screen-Shot-2018-03-10-at-12.25.43-PM.png)

我们可以通过构建一个损失项来实现这一点，该损失项惩罚我们的隐藏层激活相对于输入训练示例的大导数，本质上是惩罚输入的小变化导致编码空间大变化的实例。

用更高级的数学术语来说，我们可以将我们的正则化损失项制作为雅可比矩阵 J 的平方 Frobenius 范数 ${\left\lVert A \right\rVert_F}$，用于隐藏层激活相对于输入观察。 Frobenius 范数本质上是矩阵的 L2 范数，而雅可比矩阵只是表示向量值函数的所有一阶偏导数（在这种情况下，我们有一个训练样本向量）。

对于 m 个观测值和 n 个隐藏层节点，我们可以如下计算这些值。

$${\left\lVert A \right\rVert_F}= \sqrt {\sum\limits_{i = 1}^m {\sum\limits_{j = 1}^n {{{\left| {{a_{ij}}} \right|}^2}} } }$$

![](img/QQ截图20210911101643.png)

更简洁地说，我们可以将完整的损失函数定义为

$${\cal L}\left( {x,\hat x} \right) + \lambda {\sum\limits_i {\left\lVert {{\nabla _ x}a_i^{\left( h \right)}\left( x \right)} \right\rVert} ^2}$$

其中 ${{\nabla_x}a_i^{\left( h \right)}\left( x \right)}$ 定义了我们的隐藏层激活相对于输入 x 的梯度场，对所有 i 个训练样本求和。

## 总结

自编码器是一种神经网络架构，能够发现数据中的结构，以开发输入的压缩表示。通用自动编码器架构的许多不同变体存在，其目标是确保压缩表示代表原始数据输入的有意义的属性；通常，使用自动编码器时最大的挑战是让您的模型实际学习有意义且可推广的潜在空间表示。

因为自编码器学习如何基于在训练期间从数据中发现的属性（即输入特征向量之间的相关性）来压缩数据，所以这些模型通常只能重建与模型在训练期间观察到的观察类别相似的数据。

自编码器的应用包括不限于以下内容：

1. 降维

不完全自动编码器是用于降维的自动编码器。

这些可以用作降维的预处理步骤，因为它们可以执行快速准确的降维而不会丢失太多信息。

此外，像 PCA 这样的降维程序只能执行线性降维，而不完全自编码器可以执行大规模的非线性降维。

2. 数据去噪（例如图像、音频）

像去噪自动编码器这样的自动编码器可用于执行高效且高精度的图像去噪。

与传统的去噪方法不同，自动编码器不搜索噪声，它们通过学习图像的表示从提供给它们的噪声数据中提取图像。然后对表示进行解压缩以形成无噪声图像。

因此，去噪自编码器可以对无法通过传统方法去噪的复杂图像进行去噪。

3. 异常检测

不完全自编码器也可用于异常检测。

例如——考虑一个已经在特定数据集 P 上训练的自动编码器。对于为训练数据集采样的任何图像，自动编码器必然会给出低重建损失，并且应该按原样重建图像。

然而，对于训练数据集中不存在的任何图像，自动编码器无法执行重建，因为潜在属性不适用于网络从未见过的特定图像。

因此，异常图像会产生非常高的重建损失，并且可以在适当阈值的帮助下轻松识别为异常。